In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.table("workspace.bronze.erp_cust_az12")

#Trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
df.display()

#Customer ID Cleanup

In [0]:

df = df.withColumn(
    "cid",
    when(col("cid").startswith("NAS"),
           substring(col("cid"), 4,length(col("cid"))))
     .otherwise(col("cid"))
)

In [0]:
df.display()

#Birthdate Validation

In [0]:

df = df.withColumn(
    "bdate",
     when(col("bdate") > current_date(), None)
     .otherwise(col("bdate"))
)

In [0]:

df = df.withColumn(
    "gen",
    when(upper(col("gen")).isin("F", "FEMALE"), "Female")
     .when(upper(col("gen")).isin("M", "MALE"), "Male")
     .otherwise("n/a")
)

In [0]:
RENAME_MAP = {
    "cid": "customer_number",
    "bdate": "birth_date",
    "gen": "gender"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)
     

In [0]:
df.display()

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customers")

In [0]:

%sql
SELECT * FROM workspace.silver.erp_customers LIMIT 10